In [1]:
import numpy as np

from data_cube import DataCube
from ssm import SSM

import matplotlib.pyplot as plt

In [ ]:
### RMS SMOOTHING ###
def rms_smooth(series, N, ss):
    """
    series - time series (numpy array of dim 2 including time index as first col)
    N - number of samples in time window
    ss - step size
    """
    n_slides = (series[:, 1].size - (N - ss)) / ss # number of window slides
    res_sz = int(n_slides) # truncate number of slides for resulting vector size
    V = np.zeros(res_sz * 2).reshape(-1, 2)
    s = 0 # window start
    e = s + N # window end
    for n, i in enumerate(V):
        V[n, 0] = series[e, 0]
        V[n, 1] = (sum(series[s:e, 1]**2)/N)**(0.5)
        s += ss
        e += ss
    
    return V

---

In [2]:
dc = DataCube(
    subjects=["10","20"],
    gestures=["1", "2", "3", "4"],
    data_grp="subsample"
)
dc.load_data()

In [4]:
dc.rms_smooth(20, 5)

IndexError: index 200 is out of bounds for axis 0 with size 200

In [ ]:
tbs1 = dc.data_set["10"]["1_0_1"][:, 0:2]
#tbs1 = dc.data_set["10"]["1_0_1"][:, 0:2]

In [ ]:
tbs1.shape

In [ ]:
smt1 = rms_smooth(tbs1, 20, 5)

In [ ]:
#plt.figure()
plt.subplot(211)
plt.plot(tbs1[:,0], tbs1[:,1])
plt.subplot(212)
plt.plot(smt1[:, 0], smt1[:, 1])
plt.show()

In [ ]:
ed = lambda i, j: (i-j)**2
s1g102 = SSM(dc.data_set["10"]["1_0_1"][:, :2], metric=ed) # exclude label col
s1g102.normalize_SSM()
s1g102.calc_SSM()

In [ ]:
s1g102.plot_SSM(0)

In [ ]:
# After RMS smoothing
s1g102 = SSM(np.c_[smt1[:, 0], smt1[:, 1]], metric=ed) # exclude label col
s1g102.normalize_SSM()
s1g102.calc_SSM()
s1g102.plot_SSM(0)

---

### TS Learn

In [ ]:
import numpy
import matplotlib.pyplot as plt

from tslearn.generators import random_walks
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from tslearn import metrics

In [ ]:
ts1  = dc.data_set["10"]["1_0_1"][:, 0]
ts1 = ts1.astype(np.int32)
ts2 = dc.data_set["10"]["1_0_1"][:, 0]
ts2 = ts2.astype(np.int32)
t1 = dc.data_set["10"]["1_0_1"][:, 1]
t2 = dc.data_set["10"]["1_0_1"][:, 2]

In [ ]:
#plt.figure()
plt.subplot(211)
plt.plot(ts1, t1)
plt.subplot(212)
plt.plot(ts2, t2)
plt.show()

In [ ]:
numpy.random.seed(0)
n_ts, sz, d = 2, 100, 1
dataset = random_walks(n_ts=n_ts, sz=sz, d=d)

scaler = TimeSeriesScalerMeanVariance(mu=0., std=1.)  # Rescale time series
dataset_scaled = scaler.fit_transform(dataset)

path, sim = metrics.dtw_path(t1, t2)

In [ ]:
matrix_path = numpy.zeros((ts1.size, ts2.size), dtype=numpy.int)
for i, j in path:
    matrix_path[i, j] = 1

In [ ]:
plt.imshow(matrix_path)
plt.show

In [ ]:
plt.figure()

plt.subplot2grid((1, 3), (0, 0), colspan=2)
plt.plot(ts1, t1)
plt.plot(ts2, t2)
#plt.subplot(1, 3, 3)
#plt.imshow(matrix_path, cmap="gray_r")

plt.tight_layout()
plt.show()

In [ ]:
sim